In [13]:
import pandas as pd
insurance = pd.read_csv('insurance.csv')


In [14]:
#q1a: recode the variable
import numpy as np
insurance['east_west'] = np.where((insurance['region']=='southwest') | (insurance['region']=='northwest'), 1, 0)
#recode the categorical variables while we're at it
dummies = pd.get_dummies(insurance[['sex','smoker']],
                         dtype=int) #this turns them into 0s and 1s instead of true and false
insurance = pd.concat([insurance,dummies],axis=1)
insurance.head()

,age,sex,bmi,children,smoker,region,charges,east_west,sex_female,sex_male,smoker_no,smoker_yes
0,19,female,27.900,0,yes,southwest,16884.92400,1,1,0,0,1
1,18,male,33.770,1,no,southeast,1725.55230,0,0,1,1,0
2,28,male,33.000,3,no,southeast,4449.46200,0,0,1,1,0
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1,1,0
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1,1,0


In [15]:
#q1b: check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif #import the VIF function
from statsmodels.tools.tools import add_constant #import the add_constant function

X = insurance[['age','bmi','children','east_west','sex_female','smoker_yes']] #select the relevant variables
X = add_constant(X) #add a constant to the variables. VIF WON'T WORK CORRECTLY IF YOU DON'T DO THIS

#Create a dataframe that contains the VIF values
VIF_data = pd.DataFrame() #create a pandas dataframe
VIF_data["variable"] = X.columns #create a variable that simply lists the names of the variables you picked
VIF_data["VIF"] = [vif(X,i) 
                   for i in range(len(X.columns))] #a loop that creates vif values for each variable and then puts it in the VIF column
print(VIF_data) #print the results


     variable        VIF
0       const  36.154016
1         age   1.015475
2         bmi   1.030329
3    children   1.003994
4   east_west   1.021418
5  sex_female   1.008887
6  smoker_yes   1.010455


In [16]:
#q1c
import statsmodels.formula.api as sm
model1a = sm.ols('charges~east_west', data = insurance).fit()
print(model1a.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     6.829
Date:                Fri, 28 Mar 2025   Prob (F-statistic):            0.00907
Time:                        00:28:27   Log-Likelihood:                -14474.
No. Observations:                1338   AIC:                         2.895e+04
Df Residuals:                    1336   BIC:                         2.896e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.411e+04    460.687     30.627      0.0

In [17]:
#q1d: 
model1b = sm.ols('charges~east_west+age',data=insurance).fit()
print(model1b.summary()) #east_west stays significant
model1c = sm.ols('charges~east_west+sex_female',data=insurance).fit()
print(model1c.summary()) #east_west stays significant
model1d = sm.ols('charges~east_west+bmi',data=insurance).fit()
print(model1d.summary()) #east_west is no longer significant
model1e = sm.ols('charges~east_west+children',data=insurance).fit()
print(model1e.summary()) #east_west stays significant
model1f = sm.ols('charges~east_west+smoker_yes',data=insurance).fit()
print(model1f.summary()) #east_west is no longer significant

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     69.95
Date:                Fri, 28 Mar 2025   Prob (F-statistic):           1.29e-29
Time:                        00:28:27   Log-Likelihood:                -14411.
No. Observations:                1338   AIC:                         2.883e+04
Df Residuals:                    1335   BIC:                         2.884e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4013.5035    981.395      4.090      0.0

In [18]:
#q1d(continued)
#table showing averages split up by east_west show that the bmi in the west is lower and that fewer people smoke in the west
print(insurance[['bmi','smoker_yes','east_west']].groupby('east_west').mean())

                bmi  smoker_yes
east_west                      
0          31.38633    0.229651
1          29.89820    0.178462


In [19]:
#Exercise 2

#Load the files and import the data (importing the data takes a few seconds)
import numpy as np
import pandas as pd
AB_NYC_2019 = pd.read_excel('AB_NYC_2019.xlsx')
AB_NYC_2019.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,last_review,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,reviews_per_month,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private_room,2018-10-19,149,1,9,6,365,0.21,NaN,NaN,152.720687
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire_home_apt,2019-05-21,225,1,45,2,355,0.38,NaN,NaN,106.000000
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private_room,NaT,150,3,0,1,365,NaN,NaN,NaN,NaN
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire_home_apt,2019-07-05,89,1,270,1,194,4.64,NaN,NaN,NaN
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire_home_apt,2018-11-19,80,10,9,1,0,0.10,NaN,NaN,NaN


In [20]:
#1 make dummies
dummies = pd.get_dummies(AB_NYC_2019[['neighbourhood_group','room_type']], dtype=int)
AB_NYC_2019 = pd.concat([AB_NYC_2019,dummies],axis=1)
AB_NYC_2019.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,last_review,...,Unnamed: 17,Unnamed: 18,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten_Island,room_type_Entire_home_apt,room_type_Private_room,room_type_Shared_room
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private_room,2018-10-19,...,NaN,152.720687,0,1,0,0,0,0,1,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire_home_apt,2019-05-21,...,NaN,106.000000,0,0,1,0,0,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private_room,NaT,...,NaN,NaN,0,0,1,0,0,0,1,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire_home_apt,2019-07-05,...,NaN,NaN,0,1,0,0,0,1,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire_home_apt,2018-11-19,...,NaN,NaN,0,0,1,0,0,1,0,0


In [21]:
#2 check for multicollinearity (there is high multicollinearity with neighbourhood group when certain combinations are used?)

X = AB_NYC_2019[['latitude',
 'longitude','minimum_nights','number_of_reviews','calculated_host_listings_count','availability_365','reviews_per_month',
      'neighbourhood_group_Bronx', 'neighbourhood_group_Brooklyn', 'neighbourhood_group_Staten_Island', 'neighbourhood_group_Queens',
       'room_type_Entire_home_apt','room_type_Shared_room']].dropna() #drop missings
X = add_constant(X) #add a constant to the variables. VIF WON'T WORK CORRECTLY IF YOU DON'T DO THIS
#Create a dataframe that contains the VIF values
VIF_data = pd.DataFrame() #create a pandas dataframe
VIF_data["variable"] = X.columns #create a variable that simply lists the names of the variables you picked
VIF_data["VIF"] = [vif(X,i) 
                   for i in range(len(X.columns))] #a loop that creates vif values for each variable and then puts it in the VIF column
print(VIF_data) #print the results

                             variable           VIF
0                               const  9.492811e+06
1                            latitude  2.782603e+00
2                           longitude  2.608746e+00
3                      minimum_nights  1.042629e+00
4                   number_of_reviews  1.473929e+00
5      calculated_host_listings_count  1.081853e+00
6                    availability_365  1.143973e+00
7                   reviews_per_month  1.504758e+00
8           neighbourhood_group_Bronx  1.298890e+00
9        neighbourhood_group_Brooklyn  3.055096e+00
10  neighbourhood_group_Staten_Island  1.291000e+00
11         neighbourhood_group_Queens  2.682685e+00
12          room_type_Entire_home_apt  1.081392e+00
13              room_type_Shared_room  1.031752e+00


In [22]:
#3 standardize
from sklearn.preprocessing import StandardScaler
AB_NYC_2019_s = AB_NYC_2019
AB_NYC_2019_s[['latitude',
               'longitude',
               'minimum_nights',
               'number_of_reviews',
               'calculated_host_listings_count',
               'availability_365','reviews_per_month']] = StandardScaler().fit_transform(AB_NYC_2019_s[['latitude',
                                                                                                        'longitude',
                                                                                                        'minimum_nights',
                                                                                                        'number_of_reviews',
                                                                                                        'calculated_host_listings_count',
                                                                                                        'availability_365',
                                                                                                        'reviews_per_month']])
AB_NYC_2019_s.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,last_review,...,Unnamed: 17,Unnamed: 18,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten_Island,room_type_Entire_home_apt,room_type_Private_room,room_type_Shared_room
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,-1.493849,-0.437652,Private_room,2018-10-19,...,NaN,152.720687,0,1,0,0,0,0,1,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,0.452436,-0.684639,Entire_home_apt,2019-05-21,...,NaN,106.000000,0,0,1,0,0,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,1.468399,0.222497,Private_room,NaT,...,NaN,NaN,0,0,1,0,0,0,1,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,-0.803398,-0.164450,Entire_home_apt,2019-07-05,...,NaN,NaN,0,1,0,0,0,1,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,1.275660,0.177216,Entire_home_apt,2018-11-19,...,NaN,NaN,0,0,1,0,0,1,0,0


In [23]:
#Build the regression model
import statsmodels.formula.api as sm
#The many variable names make the code very long. You can put it on several lines though, but every line need to be surrounded by '' as you can see below
model1 = sm.ols('price~latitude+longitude+minimum_nights+number_of_reviews'
                '+calculated_host_listings_count+availability_365+reviews_per_month'
                '+ neighbourhood_group_Bronx+neighbourhood_group_Brooklyn+neighbourhood_group_Queens'
                '+neighbourhood_group_Staten_Island+room_type_Entire_home_apt+room_type_Shared_room',
               data = AB_NYC_2019_s).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     399.8
Date:                Fri, 28 Mar 2025   Prob (F-statistic):               0.00
Time:                        00:28:35   Log-Likelihood:            -2.5788e+05
No. Observations:               38843   AIC:                         5.158e+05
Df Residuals:                   38829   BIC:                         5.159e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

In [24]:
#Regression model in APA
#Making the model better
#More details: https://github.com/mwburke/stargazer/blob/master/examples.ipynb
import pandas as pd
from stargazer.stargazer import Stargazer, LineLocation
from IPython.core.display import HTML

Table = Stargazer([model1])
Table.title('Model predicting AirBnB price') #Give it a title   
Table.custom_columns(['Model']) #Give the models names
Table.show_model_numbers(False) #Remove model number
Table.significant_digits(2) #Change decimals to 2
Table.rename_covariates({'availability_365':'Year-round availability', #Rename relevant Variables
                        'calculated_host_listings_count':'Total number of listings by host',
                        'latitude': 'Latitude',
                        'longitude':'Longitude',
                        'minimum_nights':'Minimum number of nights',
                        'neighbourhood_group_Bronx':'Bronx',
                        'neighbourhood_group_Brooklyn':'Brooklyn',
                        'neighbourhood_group_Queens':'Queens',
                        'neighbourhood_group_Staten_Island':'Staten Island',
                        'number_of_reviews':'Number of reviews',
                        'reviews_per_month':'Reviews per month',
                        'room_type_Entire_home_apt':'Entire home or apartment',
                        'room_type_Shared_room':'Shared room'})
Table.add_custom_notes(['All continuous variables have been standardized']) #Add note about standardization
Table.custom_note_label('') #Remove the word "Note:" 
Table.show_degrees_of_freedom(False) #Remove the degrees of freedom 

HTML(Table.render_html()) #Render in HTML

ModuleNotFoundError: No module named 'stargazer'